# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.


### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


In [7]:
import pandas as pd
import numpy as np
import math
import scipy.stats as st

In [8]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')
#data.columns
#data.head()

In [9]:
# number of callbacks for black-sounding names
b_callbacks = sum(data[data.race=='b'].call)
b_no_callbacks = len(data[data.race == 'b'] )- sum(data[data.race=='b'].call)
w_callbacks = sum(data[data.race=='w'].call)
w_no_callbacks = len(data[data.race == 'w'] )- sum(data[data.race=='w'].call)
print('callbacks for b names: ' + str(sum(data[data.race=='b'].call)))
print('NO callbacks for b names: ' + str(len(data[data.race == 'b'] )- sum(data[data.race=='b'].call)))
print('callbacks for w names: ' + str(sum(data[data.race=='w'].call)))
print('No callbacks for w names: ' + str(len(data[data.race == 'w']) - sum(data[data.race=='w'].call)))

callbacks for b names: 157.0
NO callbacks for b names: 2278.0
callbacks for w names: 235.0
No callbacks for w names: 2200.0




**1)** In order to test if there is a correlation between race and likelihood of a callback I am going to investigate the difference of means. I will use a Z test for the difference of means because the number of samples is >30.
We can set the problem up as two Bernoulli distributions. In other words I am going to completely separate the data for 'b' names and data for 'w' names. For each I am going to split the results into two categories: 'call' or 'no call.' I will assign the 'call' category as 1 and the 'no call' category as 0. Using this I can find the mean proportion for each sample (w and b). The mean can be calculated from the following equation:

                                sample mean = [(# 'no call')*0 + (# 'call')*1]/total
                                
For each sample the total number of people is 2435. The sample mean for the 'b' names was found to be 0.0645 and for the 'w' names was 0.0965.
In order to calculate the standard deviation (std) of each sample we need to use the equation for variance, shown below.

                             variance = sum( square distance to the mean for each value)/total*
                             
If we take the square root of the variance we get the standard deviation.
    -For the 'b' group: variance = 0.0603 and std = 0.246, for the 'w' group: variance = 0.0872 and std = 0.2953.
    


In [10]:
# We can use a Bernoulli distribution where callback = 1 and no callback = 0 to find the sample mean proportion.

sample_mean_b = (b_no_callbacks*0 + b_callbacks*1)/len(data[data.race == 'b'] )
sample_mean_w = (w_no_callbacks*0 + w_callbacks*1)/len(data[data.race == 'w'] )
print('Mean Proportion b list: ' + str(sample_mean_b))
print('Mean Proportion w list: ' + str(sample_mean_w))

var_b = (b_callbacks*(1-sample_mean_b)**2 + b_no_callbacks*(0-sample_mean_b)**2)/(len(data[data.race == 'b'])-1)
var_w = (w_callbacks*(1-sample_mean_w)**2 + w_no_callbacks*(0-sample_mean_w)**2)/(len(data[data.race == 'w'])-1)
std_b = math.sqrt(var_b)
std_w = math.sqrt(var_w)

print('std_b: ' + str(std_b))
print('std_w: ' + str(std_w))

Mean Proportion b list: 0.064476386037
Mean Proportion w list: 0.0965092402464
std_b: 0.24565008364706123
std_w: 0.2953489980097223




**2)**  
    **Ho:** There is no difference in the mean proportion of callbacks for 'b' names and 'w' names. **Ha:** There is a difference in the mean proportion of callbacks for 'b' names and 'w' names.



**3)** I am going to use a margin of error of 5%. I will use a two tailed test because we are interested in whether race plays any role, not just if 'w' has the advantage. With a margin of error of 5% my critical z-value is 1.96. 
-Next I will construct a 95% confidence interval. In order to calculate my confidence interval I need the standard deviation of the difference in means which can be calculated from the equation below,

                            std(difference in means) = sqrt(variance('b')/n + variance('w')/n)
                            
where n is the number of samples (equal to 2435 here). The calculated std(difference in means) = 0.00598. From my critical z-value (z = 1.96), I can be confident that there is a 95% chance that the the true population difference in means is within -0.117 and 0.117 of our sample difference in means (found to be 0.0320).

-The **p-value** associated with my sample difference iin means can be calculated as the difference in means divided by the std(difference in means). Another way to put this is P-value = 0.0320/.00778. Therefore my P-value is 4.114 which is close to a probabilty of 0 (actual value in code below). What this means is that there is almost a 0% chance that I would observe this difference in means from a sample if the true population difference in means was 0. To put it another way, there is almost a 0% chance that a race label does not play a roll in the likelihood of getting a callback.



In [11]:
#This section shows the calculation of my 95% confidence interval as well as the p-value associated with the sample difference in means.

sample_diff_means = sample_mean_w - sample_mean_b
print('Sample Difference in Mean Proportions: ' + str(sample_diff_means))
z_critical = 1.96 #This value can be changed depending on the desired significance level.
est_pop_diff_mean = 0 #This value can be changed depending on the desired hypothesis test.

sigma_xbar_diff = math.sqrt((var_b + var_w)/len(data[data.race == 'b'] ))
print('std(sampling_distribution): ' + str(sigma_xbar_diff))

confidence = round(1.96*sigma_xbar_diff, 4)

print('I am confident that there is a 95% chance that the true population difference in mean proportion is within ' + str(confidence) + ' of our sample difference in mean proportions: ' + str(round(sample_diff_means,4)))

#Below I calculate the p-value for my sample difference in means proportion.

sample_diff_z_value = abs(sample_diff_means - est_pop_diff_mean) / sigma_xbar_diff
print('z-sample: ' + str(sample_diff_z_value))
p_value = 1 - st.norm.cdf(sample_diff_z_value)
print('p-value for the sample difference in mean proportion: ' + str(p_value))

Sample Difference in Mean Proportions: 0.0320328542094
std(sampling_distribution): 0.007784969307159196
I am confident that there is a 95% chance that the true population difference in mean proportion is within 0.0153 of our sample difference in mean proportions: 0.032
z-sample: 4.11470526672
p-value for the sample difference in mean proportion: 1.93837221231e-05




**4)** Given data on resumes randomly labeled with a race as 'w' or 'b' the statistical significance of the impact of race on the likelihood of getting a call back was investigated. This was done by looking at the difference in mean proportion of callbacks for the 'w' group and the 'b' group. Our null hypothesis was that the difference in means is 0, meaning there is no impact of race. The alternative hypothesis is that the difference in means is not 0, showing that race does have an impact. I did my calculations with a 5% margin of error meaning that I am willing to risk a Type I error 5% of the time. Another way to put this is that my significance level is 5%, so I am looking to see if the results of the study described here are more extreme than a critical value encompassing 95% of data, assuming the null hypothesis is true. What I found is that my sample difference in means is 0.0320 and the sampling distribution of difference in means (assumed to have a mean value of 0) has a std of 0.00778 (the calculations are described above). Using a two tailed z-test with 5% significance my critical z-value is found to be 1.96. This means that it should be expected that 95% of sample difference in means should lie within 1.96 standard deviations of 0. However, my sample difference in means sits at 4.114 standard deviations from 0. The likelihood the difference in means observed from this sample sitting at such an extreme value happening by chance is very low, with a P-value of almost 0. This means that the observed difference in means is not a fluke. Because of this, **I will reject the null hypothesis and will accept the hypothesis that the label of race does have an impact on the likelihood of getting a call back on a resume.**

In [12]:
#This section shows the calculation of my z-value and the results of the hypothesis test.
#Ho: The difference in means equals 0. Ha: The difference in means does not equal 0.
# Assume the null hypotheis is true. I will use a 5% significance level

sample_diff_means = sample_mean_w - sample_mean_b
z_critical = 1.96 #This value can be changed depending on the desired significance level.
est_pop_diff_mean = 0 #This value can be changed depending on the desired hypothesis test.

sigma_xbar_diff = math.sqrt((std_b**2 + std_w**2)/len(data[data.race == 'b'] ))
print('std(sampling_distribution): ' + str(sigma_xbar_diff))

sample_diff_z_value = abs(sample_diff_means - est_pop_diff_mean) / sigma_xbar_diff
print('z-sample: ' + str(sample_diff_z_value))

if sample_diff_z_value > z_critical:
    print('Conclusion: We will reject Ho and accept the alternative hypothesis. We have sufficient evidence to say that presummed race plays a role in likelihood of callbacks.')
else: 
    print('Conclusion: We do not have sufficient evidence to reject the null hypothesis, and therefore cannot say that race plays a role in likelihood of callbacks.')
    

std(sampling_distribution): 0.007784969307159197
z-sample: 4.11470526672
Conclusion: We will reject Ho and accept the alternative hypothesis. We have sufficient evidence to say that presummed race plays a role in likelihood of callbacks.





**5)** It is true that the data shows that there is a statistically significant correlation between race and the likelihood of getting a call back, but there are many more factors that could be affecting these results. Race is only one parameter of the many that are listed in the data set. It is possible that race is the most noteable factor, however we cannot know this until we test all of the other reasonable factors. One important factor to look at is the level of education, because most jobs have minimum education requirements. If I were to ammend my analysis I would start by looking individually at race, years of education, and number of previous jobs. After the individual analysis I would combine resume information to look at how education affected the outcome given their race, etc.